In [ ]:
import torch as T
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
from tqdm import tqdm
import json
import pandas as pd
import copy

In [ ]:
# Paths
path = 'E:/Repos/Cloud-Type-Identifier/Images/'
with open(f'{path}types.json', 'r') as f:
    classes = json.load(f)

class_folders = [f'{path}{c}/' for c in classes]

label2idx = {c: i for i, c in enumerate(classes)}
idx2label = {i: c for i, c in enumerate(classes)}

In [ ]:
# Create a df with all the image file paths and their labels
df = pd.DataFrame(columns=['path', 'label'])

for i, folder in enumerate(class_folders):
    for file in os.listdir(folder):
        df = df.append({'path': folder+file, 'label': i}, ignore_index=True)

# Shuffle the df
df = df.sample(frac=1).reset_index(drop=True)

# Split the df into train and test
train_df = df.iloc[:int(len(df)*0.8)]
test_df = df.iloc[int(len(df)*0.8):]

In [ ]:
# Transformations
train_transform = A.Compose([
    A.SmallestMaxSize(300),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=360, p=0.5),
    A.RandomCrop(height=224, width=224),
    A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.MultiplicativeNoise(multiplier=[0.5, 2], per_channel=True, p=0.2),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.229, 0.225]),
    A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5),
    ToTensorV2(),
])

test_transform = A.Compose([
    A.SmallestMaxSize(max_size=350),
    A.CenterCrop(height=256, width=256),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

In [ ]:
# Dataset class
class CloudDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['path']
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        label = self.df.iloc[idx]['label']

        if self.transform is not None:
            img = self.transform(image=img)['image']

        return img, label        

In [ ]:
# Create the datasets
train_dataset = CloudDataset(train_df, transform=train_transform)
test_dataset = CloudDataset(test_df, transform=test_transform)

# Create the dataloaders
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Visualize some images
def visualize(dataset, samples=10, cols=5):
    dataset = copy.deepcopy(dataset)
    viz_transform = A.Compose([t for t in dataset.transform if not isinstance(t, (A.Normalize, ToTensorV2))])
    dataset.transform = viz_transform # Remove the normalization and ToTensorV2 transforms

    rows = samples//cols
    fig, ax = plt.subplots(rows, cols, figsize=(12, 8))
    for i in range(samples):
        idx = np.random.randint(1, len(dataset))
        img, label = dataset[idx]
        ax.ravel()[i].imshow(img)
        ax.ravel()[i].set_axis_off()
        ax.ravel()[i].set_title(idx2label[label])
    plt.tight_layout()
    plt.show()

visualize(train_dataset)

In [ ]:
# Create the model
class CloudCNN(nn.Module):
    def __init__(self, n_classes):
        super(CloudCNN, self).__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, 3, 1, 1), # 224x224x3 -> 224x224x32
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, 3, 1, 1), # 224x224x32 -> 224x224x64
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2, 2), # 224x224x64 -> 112x112x64

            nn.Conv2d(64, 128, 3, 1, 1), # 112x112x64 -> 112x112x128
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, 3, 1, 1), # 112x112x128 -> 112x112x128
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2, 2), # 112x112x128 -> 56x56x128

            nn.Conv2d(128, 256, 3, 1, 1), # 56x56x128 -> 56x56x256
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, 3, 1, 1), # 56x56x256 -> 56x56x256
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2, 2), # 56x56x256 -> 28x28x256

            nn.Conv2d(256, 512, 3, 1, 1), # 28x28x256 -> 28x28x512
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.Conv2d(512, 512, 3, 1, 1), # 28x28x512 -> 28x28x512
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(2, 2), # 28x28x512 -> 14x14x512

            nn.Flatten(), # 14x14x512 -> 100352
            nn.Linear(100352, 1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, n_classes)
        )

    def forward(self, x):
        return self.network(x)

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds==labels).item()/len(preds))


In [ ]:
# Validate the model